In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

In [6]:
docs = SimpleDirectoryReader("artifacts/").load_data()

In [7]:
docs

[Document(id_='f57eb259-9391-4afe-8272-4cd6a957aa66', embedding=None, metadata={'page_label': '1', 'file_name': 'Abdullah Bin Noor Tazim - CV.pdf', 'file_path': 'g:\\Ongoing Project\\RAG based Chatbot\\artifacts\\Abdullah Bin Noor Tazim - CV.pdf', 'file_type': 'application/pdf', 'file_size': 138610, 'creation_date': '2025-11-16', 'last_modified_date': '2025-10-31'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Abdullah Bin Noor Tazim\nMachine Learning Engineer\n♂phone+880-1602584611/envel⌢peabdullah_tazim@outlook.com/gl⌢bePortfolio/linkedinLinkedIn/githubGitHub\nProfessional Summary\nFinal-year Computer Science student with hands-on exp

In [9]:
from llama_index.embeddings.langchain import LangchainEmbedding

In [10]:
system_prompt = """
You are a QA asistant .
 Your goal is to answer question as accurately
 as possible based on the
 instruction and context provided .
"""

In [11]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

g:\Ongoing Project\RAG based Chatbot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [51]:
from langchain_huggingface import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings
from llama_index.embeddings.langchain import LangchainEmbedding
from sentence_transformers import SentenceTransformer


# # 1. Load embeddings from LangChain
# hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# # 2. Wrap them for LlamaIndex
# embed_model = LangchainEmbedding(hf_embed)

embed_model = HuggingFaceEmbedding(
    model_name="intfloat/e5-large-v2",
    max_length=512,
    device="cpu",  # or "cuda"
    embed_batch_size=32,  # controls batching
    # DO NOT pass normalize_embeddings here
)

# # 3. Load Ollama LLM
llm = Ollama(model="llama3.2:1b",system_prompt=system_prompt)

# 4. Load your documents
# docs = SimpleDirectoryReader("artifacts/").load_data()


Settings.chunk_size = 512
Settings.llm = llm
Settings.embed_model = embed_model

# 5. Build the vector index
index = VectorStoreIndex.from_documents(docs)
query_engine = index.as_query_engine(llm=llm)


2025-11-23 21:27:28,179 - INFO - Load pretrained SentenceTransformer: intfloat/e5-large-v2
2025-11-23 21:27:42,665 - INFO - HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"


In [15]:
!pip show llama-index


Name: llama-index
Version: 0.14.8
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: 
Author-email: Jerry Liu <jerry@llamaindex.ai>
License-Expression: MIT
Location: G:\Ongoing Project\RAG based Chatbot\venv\Lib\site-packages
Requires: llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-llms-openai, llama-index-readers-file, llama-index-readers-llama-parse, nltk
Required-by: 


In [ ]:
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.core.postprocessor import SimilarityPostprocessor

Settings.chunk_size = 1024
Settings.chunk_overlap = 64
Settings.llm_temperature = 0.2
Settings.llm = llm
Settings.embed_model = embed_model
Settings.similarity_top_k = 5
Settings.response_mode = "compact"
Settings.node_postprocessors = [SimilarityPostprocessor(similarity_cutoff=0.40)]
# Settings.text_splitter = SemanticSplitter(chunk_size=900, chunk_overlap=32)
Settings.system_prompt = "You are a factual assistant. Use only the retrieved data to answer."

In [49]:
query_engine = index.as_query_engine(llm=llm)

In [56]:
response = query_engine.query("what are the work experience mentioned in the tazim's CV?")
print(response)

2025-11-23 21:29:23,484 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


Abdullah Bin Noor Tazim has two work experiences:

1. Trainee at Neurochip Industries Ltd., Dhaka, Bangladesh (January 2024 - March 2024)
2. NLP Engineer Intern at Elvvo Pathways, Remote, Kairo, Egypt (October 2025 - Present)
